# `mkdocs` plugin for jupyter notebooks

https://www.mkdocs.org/dev-guide/plugins/
https://github.com/danielfrg/mkdocs-jupyter

- [x] add plugin to `mkdocs.yml`
- [x] define plugin entry point for `tonyfast`
- [ ] build the plugin
- [x] integrate the plugin

In [10]:
    import json, nbconvert, nbformat

In [21]:
    def prepare_notebook(nb):
        for cell in nb.cells:
            cell.source = "".join(cell.source)
            for output in cell.get("outputs", ""):
                if "text" in output:
                    output.text = "".join(output.text)
                if "data" in output:
                    for k, v in output["data"].items():
                        if k.startswith(("text",)):
                            output.data[k] = "".join(v)
                            

In [24]:
    import mkdocs.plugins
    mkdocs.utils.markdown_extensions += ".ipynb",
    class Config(mkdocs.config.base.Config):
        pass # foo = mkdocs.config.config_options.Type(str, default='a default value')
    class MarkdownNotebook(mkdocs.plugins.BasePlugin):
        exporter = nbconvert.exporters.MarkdownExporter()
        def on_page_markdown(self, markdown, page, config, files):
            pass # not the hit
            
        def on_page_read_source(self, page, config):
            if page.file.src_uri.endswith((".ipynb", )):
                return self.exporter.from_filename(page.file.abs_src_path)[0]            